In [1]:
import os, json
from pymongo import MongoClient
from dotenv import load_dotenv

load_dotenv()
client = MongoClient(os.getenv("MONGODB_URI"), tz_aware=True)
db = client[os.getenv("MONGODB_DATABASE")]
dispensers = db["dispensers"]

location = "sikatuna"

pipeline = [
    {"$match": {"location": location}},
    {
        "$lookup": {
            "from": "pumps",
            "let": {"did": "$id", "loc": "$location"},
            "pipeline": [
                {"$match": {
                    "$expr": {
                        "$and": [
                            {"$eq": [{"$toString": "$dispenser_id"}, {"$toString": "$$did"}]},
                            {"$eq": ["$location", "$$loc"]}
                        ]
                    }
                }},
                {"$project": {"_id": 0, "id": 1, "name": 1}}
            ],
            "as": "pumps"
        }
    },
    {"$project": {"_id": 0, "id": 1, "name": 1, "pumps": 1}},
    {"$sort": {"name": 1}}
]

docs = list(dispensers.aggregate(pipeline))
print(json.dumps(docs, indent=2, ensure_ascii=False))


[
  {
    "id": "1",
    "name": "dispenser-one",
    "pumps": [
      {
        "id": 1,
        "name": "premium"
      },
      {
        "id": 2,
        "name": "regular"
      }
    ]
  },
  {
    "id": "2",
    "name": "dispenser-two",
    "pumps": [
      {
        "id": 4,
        "name": "diesel"
      },
      {
        "id": 3,
        "name": "regular"
      }
    ]
  }
]
